In [2]:
from azureml.core import Workspace, Run
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.1005480


In [3]:
ws = Workspace.get_or_create(name='mladsdemo', location='eastus2', subscription_id='6a275e9b-a3e6-42c2-b158-760d27f5cb83', resource_group='mladsdemo')
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mladsdemo
Azure region: eastus2
Subscription id: 6a275e9b-a3e6-42c2-b158-760d27f5cb83
Resource group: mladsdemo


In [4]:
# get the text data from the github repo and unzip it
from fit_and_store_pipeline import unzip_file_here
import urllib
import os

if not os.path.isfile('./attack_data.csv'):
    if not os.path.isfile('./text_data.zip'): 
        urllib.request.urlretrieve('https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip', 'text_data.zip')
    unzip_file_here('text_data.zip')

if not os.path.isfile('miniglove_6B_50d_w2v.txt'):
    unzip_file_here('miniglove_6B_50d_w2v.zip')
    
print('Data files here')

Data files here


In [1]:
from fit_and_store_pipeline import *
script_main()

C:\Users\tosingli\AppData\Local\conda\conda\envs\myenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[[849  13]
 [ 98  40]]
{"result": [1, 0, 0, 0, 1]}


In [5]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy
    - scikit-learn
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core
    - gensim

Overwriting myenv.yml


In [6]:
# you may need to register the provider in your subscription 
# az provider register -n Microsoft.ContainerInstance

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 7, 
                                               tags = ['MLADS 2018 Text Workshop'], 
                                               description = 'Sample container deployment')

In [11]:
%%time
# this will take 5-10 minutes to finish
service = ws.deploy_webservice(name = 'aci-attack-clsfr2',
                               deploy_config = aciconfig,
                               target = None,
                               models = ['rf_attack_classifier_pipeline.pkl'],
                               runtime = 'python',
                               conda_file = 'myenv.yml',
                               driver = 'fit_and_store_pipeline.py')
service.wait_for_deployment(show_output = True)

Registering model rf_attack_classifier_pipeline.


Client-Request-ID=719a679c-6d3c-11e8-9eb8-dc53600a4933 Retry policy did not allow for a retry: Server-Timestamp=Mon, 11 Jun 2018 05:58:20 GMT, Server-Request-ID=c56666b5-f01e-0026-4d49-0113e4000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:c56666b5-f01e-0026-4d49-0113e4000000Time:2018-06-11T05:58:21.0346796Z</Message></Error>.
Client-Request-ID=71f13b64-6d3c-11e8-b1cf-dc53600a4933 Retry policy did not allow for a retry: Server-Timestamp=Mon, 11 Jun 2018 05:58:21 GMT, Server-Request-ID=08b52025-b01e-00a2-6c49-0145cc000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:08b52025-b01e-00a2-6c49-0145c

Creating image
Image creation operation finished for image aci-attack-clsfr2:1, operation "Succeeded"
Creating service


WebserviceException: Received bad response from Model Management Service:
Response Code: 403
Headers: {'Date': 'Mon, 11 Jun 2018 06:01:05 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'api-supported-versions': '2018-03-01-preview', 'x-ms-client-request-id': '23eff2bcb9ab4d9582957c4ea9d5e8fb', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload', 'Content-Encoding': 'gzip'}
Content: b'{"code":"Forbidden","statusCode":403,"message":"Forbidden","details":[{"code":"AuthorizationFailed","message":"The client \'8199f5b9-08d7-4e08-a2eb-6d5aaba078ea\' with object id \'8199f5b9-08d7-4e08-a2eb-6d5aaba078ea\' does not have authorization to perform action \'Microsoft.ContainerInstance/containerGroups/write\' over scope \'/subscriptions/6a275e9b-a3e6-42c2-b158-760d27f5cb83/resourceGroups/mladsdemo/providers/Microsoft.ContainerInstance/containerGroups/aci-attack-clsfr2\'."}]}'

In [17]:
# verify the deployed service
print('web service is hosted in ACI:', service.scoring_uri)

web service is hosted in ACI: http://40.76.71.6:5001/score


In [10]:
service.delete()

NameError: name 'service' is not defined

In [18]:
# Now score the webservice

import json

test_attacks = [ 'You are scum.', 'I like your shoes.', 'You are pxzx.', 
                 'Your mother was a hamster and your father smelt of elderberries',
                 'One bag of hagfish slime, please']

# score the first row from the test set.
test_samples = json.dumps({"data": test_attacks})
test_samples = bytes(test_samples, encoding = 'utf8')

service.run(input_data = test_samples)

ReadTimeout: HTTPConnectionPool(host='40.76.71.6', port=5001): Read timed out. (read timeout=80)